# NER Wikidata

## Set up

In [64]:
import pandas as pd
import urllib.request
import re 
import json

In [2]:
df = pd.read_csv("/Users/yangyuting/Library/Mobile Documents/com~apple~CloudDocs/01PhD/02ISU/04Classes/2022Fall/COM S 579X NLP/Project/wikidata-qrank.csv")

## 1. Retrieve entity names

### 1.1 Decode into bytes

In [29]:
df1 = df[:1000]

In [ ]:
df1 = df[:1000]
list1 = df1.Entity.to_list()
Name = []
for i, entity in enumerate(list1):
    print('Entity', i)
    response = urllib.request.urlopen('https://www.wikidata.org/w/api.php?action=wbgetentities&props=labels&ids='+entity+'&languages=en&format=json')
    value = response.read().decode()
    try:
        value = value.split('"value":"')[1].split('"}}}}')[0]
        Name.append(value)
    except:
        Name.append('NA')
df1.insert(1, column = 'Name', value = Name)
del i, entity, Name, list1

In [8]:
df1.query('Name == "NA"')

,Entity,Name,QRank
18,Q110918355,NA,34741612


In [6]:

response = urllib.request.urlopen('https://www.wikidata.org/w/api.php?action=wbgetentities&props=labels&ids=Q1107&languages=en&format=json')
value = response.read().decode()
print(value)
# value = value.split('"value":"')[1].split('"}}}}')[0]
# Name.append(value)

{"entities":{"Q1107":{"type":"item","id":"Q1107","labels":{"en":{"language":"en","value":"anime"}}}},"success":1}


In [5]:
list1 = df1.Entity.to_list()
list1[999]

'Q1107'

### 1.2 Decode into dictionary

In [ ]:
def top_N_NEs(input_csv_ranking_file_path: str, output_csv_file_path: str, top_N: int = 10):
    df = pd.read_csv(input_csv_ranking_file_path)
    df2 = df[:top_N]
    list2 = df2.Entity.to_list()
    Name = []
    for i, entity in enumerate(list2):
        print('Entity', i)
        response = urllib.request.urlopen('https://www.wikidata.org/w/api.php?action=wbgetentities&props=labels&ids='+entity+'&languages=en&format=json')
        value = response.read().decode('utf-8')
        value = json.loads(value)
        print(type(entity))
        try:
            value = value['entities'][entity]['labels']['en']['value']
            # value = value['entities'][f'{entity}]['labels']['en']['value']
            Name.append(value)
        except:
            Name.append('NA')
    df2.insert(1, column = 'Name', value = Name)
    df2.to_csv(output_csv_file_path)
    del i, entity, Name, list2

top_N_NEs("/Users/yangyuting/Library/Mobile Documents/com~apple~CloudDocs/01PhD/02ISU/04Classes/2022Fall/COM S 579X NLP/Project/wikidata-qrank.csv",
          "/Users/yangyuting/Library/Mobile Documents/com~apple~CloudDocs/01PhD/02ISU/04Classes/2022Fall/COM S 579X NLP/Project/wikidata-qrank_top10.csv")

In [23]:
"Q317521" "Q1107"
response = urllib.request.urlopen('https://www.wikidata.org/w/api.php?action=wbgetentities&props=labels&ids=Q317521&languages=en&format=json')
value = response.read()
value = json.loads(value.decode('utf-8'))
print(value)
value['entities']['Q317521']['labels']['en']['value']

{'entities': {'Q317521': {'type': 'item', 'id': 'Q317521', 'labels': {'en': {'language': 'en', 'value': 'Elon Musk'}}}}, 'success': 1}


'Elon Musk'

## 2. 2nd two weeks

##### Goal 0: Expand top_n_NEs above to return the top-N NEs as a list of strings

In [ ]:
def top_N_NEs(input_csv_ranking_file_path: str, top_N: int = 10):
    df = pd.read_csv(input_csv_ranking_file_path)
    df2 = df[:top_N]
    list2 = df2.Entity.to_list()
    Name = []
    for i, entity in enumerate(list2):
        print('Entity', i)
        response = urllib.request.urlopen('https://www.wikidata.org/w/api.php?action=wbgetentities&props=labels&ids='+entity+'&languages=en&format=json')
        value = response.read().decode('utf-8')
        value = json.loads(value)
        print(type(entity))
        try:
            value = value['entities'][entity]['labels']['en']['value']
            # value = value['entities'][f'{entity}]['labels']['en']['value']
            Name.append(value)
        except:
            Name.append('NA')
    return Name
    del i, entity, Name, list2

list1 = top_N_NEs("/Users/yangyuting/Library/Mobile Documents/com~apple~CloudDocs/01PhD/02ISU/04Classes/2022Fall/COM S 579X NLP/Project/wikidata-qrank.csv")

##### Goal 1: Understand how to evaluate an NER model/method based on the papers you read in Goal 2 for the first two weeks. 

SOTA performance: F1 score, precision, and recall

### Goal 2: Develop a Python function that takes a sentence as input and returns a NER tag sequence

(in an appropriate tagging schemes, such as BIO), using simple string match pseudocode below. One thing you need to think is how to match overlapping NEs, e.g., “iowa” vs. “iowa state university”. One easy heurstics is to always match the longest one.

[Named Entity Recognition Tagging](https://cs230.stanford.edu/blog/namedentity/)  
[NER model evaluation](https://towardsdatascience.com/entity-level-evaluation-for-ner-task-c21fb3a8edf)  
[Spacy custom tokenizer to include only hyphen words as tokens using Infix regex](https://stackoverflow.com/questions/51012476/spacy-custom-tokenizer-to-include-only-hyphen-words-as-tokens-using-infix-regex)

##### 1. CONLL2003 setup

In [ ]:
conll = urllib.request.urlopen("https://datasets-server.huggingface.co/splits?dataset=conll2003")
dic = conll.read().decode()

In [24]:
conll_nertag = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}

In [ ]:
def change_ner_tags(tags):
    for id, i in enumerate(tags):
        # print(tags)
        for index, value in conll_nertag.items():
            if i == value:
                tags[id] = index.split('-')[0]
    return tags

In [ ]:
conll = pd.DataFrame([
    [[ "China", "controlled", "most", "of", "the", "match", "and", "saw", "several", "chances", "missed", "until", "the", "78th", "minute", "when", "Uzbek", "striker", "Igor", "Shkvyrin", "took", "advantage", "of", "a", "misdirected", "defensive", "header", "to", "lob", "the", "ball", "over", "the", "advancing", "Chinese", "keeper", "and", "into", "an", "empty", "net", "." ], [ 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0 ]],
    [[ "Oleg", "Shatskiku", "made", "sure", "of", "the", "win", "in", "injury", "time", ",", "hitting", "an", "unstoppable", "left", "foot", "shot", "from", "just", "outside", "the", "area", "." ], [ 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ]]
],
    columns = ('tokens', 'ner_tags')
)
conll

In [83]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
conll_sen = [TreebankWordDetokenizer().detokenize(i) for i in conll.tokens.to_list()]
print(conll_sen)

conll_ner = [change_ner_tags(a) for a in conll.ner_tags.to_list()]
print(conll_ner)


['China controlled most of the match and saw several chances missed until the 78th minute when Uzbek striker Igor Shkvyrin took advantage of a misdirected defensive header to lob the ball over the advancing Chinese keeper and into an empty net.', 'Oleg Shatskiku made sure of the win in injury time, hitting an unstoppable left foot shot from just outside the area.']
[['B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'B', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]


##### 2. Tokenization setup

In [ ]:

import re
import spacy
from spacy.tokenizer import Tokenizer
from spacy.util import compile_prefix_regex, compile_infix_regex, compile_suffix_regex

def custom_tokenizer(nlp):
    infix_re = re.compile(r'''[.\,\?\:\;\...\‘\’\`\“\”\"\'~]''')
    prefix_re = compile_prefix_regex(nlp.Defaults.prefixes)
    suffix_re = compile_suffix_regex(nlp.Defaults.suffixes)

    return Tokenizer(nlp.vocab, prefix_search=prefix_re.search,
                                suffix_search=suffix_re.search,
                                infix_finditer=infix_re.finditer,
                                token_match=None)

sen = custom_tokenizer("I joined the Ph.D. program of Ivy College of Business at Iowa State University in Ames.")

In [15]:
from nltk.tokenize import word_tokenize
sen = "I'm Yuting Yang. I joined the Ph.D. program of Ivy College of Business at Iowa State University in Ames on 2022-08-15. People are open-minded here. The water costs $2.00."
print(word_tokenize(sen))

['I', "'m", 'Yuting', 'Yang', '.', 'I', 'joined', 'the', 'Ph.D.', 'program', 'of', 'Ivy', 'College', 'of', 'Business', 'at', 'Iowa', 'State', 'University', 'in', 'Ames', 'on', '2022-08-15', '.', 'People', 'are', 'open-minded', 'here', '.', 'The', 'water', 'cost', '$', '2.00', '.']


##### 3. NER tagging function

In [60]:
import re
from nltk.tokenize import word_tokenize
sen = "I joined the Ph.D. program of Ivy College of Business at Iowa State University in Ames."
NE_list = ['Iowa State University', 'College of Business','Ivy College of Business', 'Iowa', 'Ivy','Ames']

def brutal_force_ENR(sentence:str, NE_list: list[str], scheme='BIO'):
    sen_list = word_tokenize(sentence)
    # print(sen_list)
    return_list = ['O']*len(sen_list)
    # print('\nThe list to be revised: \n',return_list, '\n')
    
    pos_list = []
    for ne in NE_list:
        position_start = sentence.find(ne)
        position_end = position_start + len(ne) - 1
        l = len(ne.split())
        print(ne, position_start, position_end, l)
        pos_list.append([l, position_start])
    # print('\n The number of tokens in the entity, the starting position of the entity:\n', pos_list)
    
    str_len = 0
    id_list = []
    for id, token in enumerate(sen_list):
        str_len += len(token) + 1
        for l, pos in pos_list:
            if pos == 0:
                id_list.append([0, l])
            elif str_len == pos:
                id_list.append([id+1, l])
    # print('\nThe position in the return_list, the number of tokens to be revised\n',id_list)
    
    for id, length in id_list:
        if return_list[id] == 'I':
            pass
        else:
            return_list[id] = 'B'
            while length > 1:
                return_list[id+1] = 'I'
                length -= 1
                id += 1
    # print('\n Output: \n', sen_list, '\n', return_list)
    print('\n Tokenize: \n', sen_list)
    return return_list
# brutal_force_ENR(sen, NE_list)

##### 4. Performance function

In [18]:

# from seqeval.metrics import accuracy_score
# from seqeval.metrics import f1_score
from seqeval.metrics import classification_report
def a_judge_function(predicted_tag_seq: list[list], ground_truth_tag_seq: list[list]):
    return classification_report([ground_truth_tag_seq], [predicted_tag_seq])

In [ ]:
classification_report([['O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'I', 'I', 'O', 'B', 'I', 'I', 'O', 'O']], [['O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'I', 'I', 'O', 'B', 'I', 'I', 'O', 'B']])

### Goal 3 Test brutal_force_NER on the CoNLL2003 and collect its performance quantities on CoNLL2003.  

In [54]:
def test_on_CONLL(sentences: list[str], ground_truth_tag_sequences: list[str], NE_list: list[str], scheme="BIO"):
    for (sentence, ground_truth_tag_seq) in zip(sentences, ground_truth_tag_sequences):
        print(sentence)
        tag_seq = brutal_force_ENR(sentence, NE_list, scheme = scheme)
        print('\nGround truth\n', ground_truth_tag_seq, '\nBrutal force tagging\n', tag_seq)
        performance_matrix = a_judge_function(ground_truth_tag_seq, tag_seq)
    # return len(sentences)
    return performance_matrix, len(sentences)


In [86]:

# sen = "I joined the Ph.D. program of Ivy College of Business at Iowa State University in Ames."
# NE_list = ['Iowa State University', 'Ivy College of Business', 'Iowa', 'Ivy', 'Ames']
# true_list = ['O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'I', 'I', 'O', 'B', 'I', 'I', 'O', 'B','O']
NE_list = ['China', 'Uzbek', 'Igor Shkvyrin','Chinese']

test_on_CONLL([conll_sen[0]], [conll_ner[0]], NE_list) 

China controlled most of the match and saw several chances missed until the 78th minute when Uzbek striker Igor Shkvyrin took advantage of a misdirected defensive header to lob the ball over the advancing Chinese keeper and into an empty net.
China 0 4 1
Uzbek 93 97 1
Igor Shkvyrin 107 119 2
Chinese 205 211 1

 Tokenize: 
 ['China', 'controlled', 'most', 'of', 'the', 'match', 'and', 'saw', 'several', 'chances', 'missed', 'until', 'the', '78th', 'minute', 'when', 'Uzbek', 'striker', 'Igor', 'Shkvyrin', 'took', 'advantage', 'of', 'a', 'misdirected', 'defensive', 'header', 'to', 'lob', 'the', 'ball', 'over', 'the', 'advancing', 'Chinese', 'keeper', 'and', 'into', 'an', 'empty', 'net', '.']

Ground truth
 ['B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'B', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] 
Brutal force tagging
 ['B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',

('              precision    recall  f1-score   support\n\n           _       1.00      1.00      1.00         4\n\n   micro avg       1.00      1.00      1.00         4\n   macro avg       1.00      1.00      1.00         4\nweighted avg       1.00      1.00      1.00         4\n',
 1)